In [1]:
import torch
import torch.nn as nn
from rfft_lib import rfft_audio
import onnx
import math

In [2]:
audio_path = "/home/satya-tt0492/Documents/TT/ex-7/audio/small.wav"

In [3]:
audio = rfft_audio.load_audio(audio_path)
print(audio)

tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -6.1035e-05,
         3.0518e-05,  0.0000e+00])


In [4]:
audio.shape

torch.Size([262094])

In [5]:
def next_power_of_2_calc(n):
    if n <= 1:
        return 1
    return 2 ** (math.ceil(math.log2(n)))

In [6]:
N = audio.shape[0]
    
# Pad to the next power of 2 if necessary
if (N & (N - 1)) != 0:
    next_power_of_2 = next_power_of_2_calc(N)
    padding_size = next_power_of_2 - N
    audio = torch.cat([audio, torch.zeros(padding_size)])
    N = next_power_of_2

In [7]:
audio

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [8]:
audio.shape

torch.Size([262144])

In [9]:
class RFFTModel(nn.Module):
    def __init__(self):
        super(RFFTModel,self).__init__()
    
    def forward(self, audio):
        rfft_magnitude = rfft_audio.apply_rfft(audio)
        return rfft_magnitude

In [10]:
model = RFFTModel()

In [11]:
onnx_file_path = "simple_model.onnx"

# Export the model to ONNX
torch.onnx.export(
    model,                   # The PyTorch model to export
    audio,             # A dummy input tensor of shape [5]
    onnx_file_path,          # Path to save the exported ONNX model
    export_params=True,      # Store the learned parameters inside the model file       # The ONNX opset version to use (12 is commonly supported)
    do_constant_folding=True,# Apply constant folding optimization
    input_names=['input'],  # The name of the input tensor
    output_names=['output'],# The name of the output tensor
    dynamic_axes={           # Allow dynamic batch size
        'input': {0: 'batch_size'},  # The batch size of input can vary
        'output': {0: 'batch_size'}  # The batch size of output can vary
    }
)

/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


UnsupportedOperatorError: Exporting the operator 'aten::fft_fft' to ONNX opset version 18 is not supported. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues.

In [15]:
# onnx_program.save("rfft_model.onnx")

NameError: name 'onnx_program' is not defined

In [16]:
# onnx_model = onnx.load("rfft_model.onnx")
# onnx.checker.check_model(onnx_model)

In [30]:
import onnxruntime

In [31]:
ort_session = onnxruntime.InferenceSession("simple_model.onnx")

In [32]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [33]:
audio = torch.rand(40)

In [34]:
onnx_input = onnx_program.adapt_torch_inputs_to_onnx(audio)

NameError: name 'onnx_program' is not defined

In [21]:
onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}
onnx_output = ort_session.run(None,onnxruntime_input)

2024-11-07 12:57:04.782936510 [E:onnxruntime:, sequential_executor.cc:516 ExecuteKernel] Non-zero status code returned while running Reshape node. Name:'/Reshape_100' Status Message: /onnxruntime_src/onnxruntime/core/providers/cpu/tensor/reshape_helper.h:45 onnxruntime::ReshapeHelper::ReshapeHelper(const onnxruntime::TensorShape&, onnxruntime::TensorShapeVector&, bool) input_shape_size == size was false. The input tensor cannot be reshaped to the requested shape. Input shape:{2}, requested shape:{1}



RuntimeException: [ONNXRuntimeError] : 6 : RUNTIME_EXCEPTION : Non-zero status code returned while running Reshape node. Name:'/Reshape_100' Status Message: /onnxruntime_src/onnxruntime/core/providers/cpu/tensor/reshape_helper.h:45 onnxruntime::ReshapeHelper::ReshapeHelper(const onnxruntime::TensorShape&, onnxruntime::TensorShapeVector&, bool) input_shape_size == size was false. The input tensor cannot be reshaped to the requested shape. Input shape:{2}, requested shape:{1}


In [51]:
x = onnx_output[0]

In [52]:
y = abs(torch.fft.fft(audio)).numpy()

In [53]:
import numpy as np

In [54]:
np.mean(np.abs(x - y))

1.238659e-07